In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import scipy.io as sio
# you can use pip to install/update missing packages. 

In [2]:
## Load the data 

# Load the partially observed (Rpartial) rating matrices (xxuyyi means #xx users #yy items)
data = sio.loadmat('100u20ipartial.mat')
Rpartial = data['Rpartial']

# The data corresponds to user ratings songs from the Rolling Stone top 20.
# 1		Bob Dylan - Like a Rolling Stone
# 2		The Rolling Stones - Satisfaction
# 3		John Lennon - Imagine
# 4		Marvin Gaye - What's Going On
# 5		Aretha Franklin - Respect
# 6		The Beach Boys - Good Vibrations
# 7		Chuck Berry - Johnny B. Goode
# 8		The Beatles - Hey Jude
# 9		Nirvana - Smells Like Teen Spirit
# 10	Ray Charles - What'd I Say (Parts 1 And 2)
# 11	The Who - My Generation
# 12	Sam Cooke - A Change Is Gonna Come
# 13	The Beatles - Yesterday
# 14	Bob Dylan - Blowin' in the Wind
# 15	The Clash - London Calling
# 16	The Beatles - I Want to Hold Your Hand
# 17	Jimi Hendrix - Purple Haze
# 18	Chuck Berry - Maybellene
# 19	Elvis Presley - Hound Dog
# 20	The Beatles - Let It Be

# The rating is on a scale from 1 (very bad) to 5 (very good). A 0 means
# the entry is missing.

# infer the size of the matrix (m x n)
[m,n] = np.shape(Rpartial)

# Eric likes Jimi Hendrix (song 17), what do you suggest for Eric? 
# Start by adding the user data of Eric to Rpartial
R_Eric = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0,])
# Redefine Rpartial (and its dimension)
Rpartial = np.vstack((Rpartial,R_Eric))
m += 1

In [6]:
## Question 5 (use cvxpy, make sure to use cp. and avoid np.)

# Define the decision variables: 
# Note! X is not symmetric. 
# Without the 'full' constraint: wrong. 
        
X = cp.Variable((m, n))
Lambda11 = cp.Variable((n, n), symmetric=True)
Lambda22 = cp.Variable((m, m), symmetric=True)

# Define the objective function:
# Note! You cannot always group the Lambda matrices (dimension might differ).
obj = 0.5 * (cp.trace(Lambda11) + cp.trace(Lambda22))

# Define the constraints:
# Constraints enforcing observed ratings (use Rpartial) 
# that is, if Rij==0, the value is "not observed". 
observed = Rpartial != 0
constr = []
constr.append(X[observed] == Rpartial[observed])

# Define the constraints enforcing the range beween 1 and 5
constr.append(X >= 1)
constr.append(X <= 5)
# Define the semidefinite constraint 
bmat = cp.bmat([[Lambda11, -X.T], [-X, Lambda22]])
constr.append(bmat >> 0)

In [8]:
## Solve the SDP

# Make sure you have a SDP solver   
# 100u20i can take a few minutes (tic-toc measures the time) 
# Set 'verbose' to 1 if you want to see the progress
prob = cp.Problem(cp.Minimize(obj), constr)
prob.solve(solver=cp.MOSEK, verbose=True)


                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Dec 08 01:38:41 AM: Your problem has 12621 variables, 4 constraints, and 0 parameters.
(CVXPY) Dec 08 01:38:41 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 08 01:38:41 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 08 01:38:41 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 08 01:38:41 AM: Compiling problem (target solver=MOSEK).
(CVXPY) Dec 08 01:38:41 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffin

217.3691532457823

In [13]:
a = 0.45
np.ceil(a)

1.0

In [14]:
# Retrieve and round the optimal solution
# Convince yourself of the validity of the rounding. 
Xopt_real = X.value
Xopt_int = np.ceil(Xopt_real)

# print the recommendation to Eric
X_Eric = Xopt_int[m-1,:]
print(X_Eric)

[4. 4. 2. 3. 2. 2. 3. 2. 4. 3. 4. 3. 2. 4. 4. 3. 5. 2. 2. 3.]
